Analyse des données abérrantes 

Pour l'analyse des données aberrantes, nous avons commencé par essayer des méthodes classiques avec les quantiles et le score-z. Les résultats étant trop différents d'un type à l'autre, nous avons plutôt opté pour une régression linéaire avec un seuil choisi. Ce choix de régression linéaire a été fait car le graphique suit une tendance linéaire.

On commence par trouver les indices (lignes) des données aberrantes avec la méthode get_outliers_ind_regression_lin :

In [ ]:
function get_outliers_ind_regression_lin(data, x_col, y_col; threshold=2.5)
    slope, intercept = linear_regression(data, x_col, y_col)
    
    x = data[!, x_col]
    y = data[!, y_col]
    
    y_pred = slope .* x .+ intercept
    
    residuals = abs.(y .- y_pred)
    
    residuals_std = std(residuals)
    
    outlier_indices = findall(residuals .> threshold * residuals_std)
    
    outliers = data[outlier_indices, :]
    
    return outlier_indices
end

Ensuite, une méthode pour obtenir ces données aberrantes seulement (get_outliers_regression_lin) :

In [ ]:
function get_outliers_regression_lin(data, outliers_indices)
    outliers = data[outliers_indices, :]
    return outliers
end;

De plus, une méthode pour les séparer des données régulières (remove_outliers_regression_lin):

In [ ]:
function remove_outliers_regression_lin(data::DataFrame, x_col::Symbol, y_col::Symbol; threshold=2.5)
    outlier_indices = get_outliers_ind_regression_lin(data, x_col, y_col, threshold=threshold)

    # println("Outlier Indices Identified: ", outlier_indices)

    keep_mask = trues(nrow(data))
    
    if !isempty(outlier_indices)
        keep_mask[outlier_indices] .= false
    end

    cleaned_data = data[keep_mask, :]
    
    return cleaned_data
end

Dernièrement, une méthode pour visualer les données avant et après le retrait des données aberrantes (plot_outliers):

In [ ]:

 function plot_outliers(uniqueD)
 # Step 2: Find and Display Outliers with Regression
 outliers_indices = get_outliers_ind_regression_lin(uniqueD, :cylindree, :consommation)
 outliers_regression = get_outliers_regression_lin(uniqueD, outliers_indices)

 # Step 3: Plot Original Data, Regression Line, and Outliers
 slope, intercept = linear_regression(uniqueD, :cylindree, :consommation)
 regression_line_y = slope .* uniqueD.cylindree .+ intercept

 layer_original = layer(x=uniqueD.cylindree, y=uniqueD.consommation, color=uniqueD.type, Theme(default_color="blue"))
 layer_linear_regression = layer(x=uniqueD.cylindree, y=regression_line_y, Geom.line, Theme(default_color="green"))
 layer_regression_outliers = layer(x=outliers_regression.cylindree, y=outliers_regression.consommation, Geom.point, Theme(default_color="red"))

 
 display(plot(layer_regression_outliers, layer_original, layer_linear_regression, Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Original Data with Outliers")))

 # Step 4: Remove Outliers Based on Regression Line
 cleaned_data = remove_outliers_regression_lin(uniqueD, :cylindree, :consommation, threshold=2.5)

 # Step 5: Plot Cleaned Data with New Regression Line
 slope_cleaned, intercept_cleaned = linear_regression(cleaned_data, :cylindree, :consommation)
 regression_line_y_cleaned = slope_cleaned .* cleaned_data.cylindree .+ intercept_cleaned

layer_cleaned = layer(x=cleaned_data.cylindree, y=cleaned_data.consommation, color=cleaned_data.type, Theme(default_color="blue"))
layer_linear_regression_cleaned = layer(x=cleaned_data.cylindree, y=regression_line_y_cleaned, Geom.line, Theme(default_color="green"))

# Display plot for cleaned data
display(plot(layer_cleaned, layer_linear_regression_cleaned, 
             Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Cleaned Data with Regression Line")))

end